In [1]:
import mido 
from mido import MidiFile
import py_midicsv as pm

import numpy as np
import pandas as pd

In [2]:
msg = mido.Message('note_on', note=60)
msg.copy(channel=2)

Message('note_on', channel=2, note=60, velocity=64, time=0)

In [3]:
md = MidiFile('/home/ivan/Desktop/Notes/MIDI/35.1.mid')

In [4]:
md.tracks[0]

MidiTrack([
  MetaMessage('track_name', name='Pianoteq 6.7', time=0),
  MetaMessage('instrument_name', name='Steinway D Prelude', time=0),
  MetaMessage('sequencer_specific', data=(153, 36, 118, 133, 1, 0, 0, 0, 0, 0, 192, 44, 22, 176, 216, 65), time=0),
  MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('set_tempo', tempo=454545, time=0),
  Message('control_change', channel=0, control=64, value=127, time=0),
  Message('note_on', channel=0, note=43, velocity=80, time=4902),
  Message('note_on', channel=0, note=67, velocity=77, time=32),
  Message('note_on', channel=0, note=67, velocity=0, time=328),
  Message('note_on', channel=0, note=50, velocity=63, time=313),
  Message('note_on', channel=0, note=55, velocity=63, time=444),
  Message('note_on', channel=0, note=67, velocity=74, time=125),
  Message('note_on', channel=0, note=50, velocity=0, time=2),
  Message('note_on', channel=0, note=43, velocity=0,

In [5]:
track = md.tracks[0]

In [6]:
track[-3]

Message('note_on', channel=0, note=67, velocity=0, time=169)

In [7]:
from midi_utils import get_midi_data

midi_data = get_midi_data(track)
midi_data.shape


(2342, 4)

In [8]:
midi_data[:15]

,on,note,velocity,time
0,True,43,80,4902
1,True,67,77,32
2,False,67,0,328
3,True,50,63,313
4,True,55,63,444
5,True,67,74,125
6,False,50,0,2
7,False,43,0,146
8,False,55,0,33
9,False,67,0,26


In [9]:
from midi_utils import _transform_data


df = _transform_data(midi_data)
df

,note,velocity,time,dur
0,43,80,4902,1390
1,67,77,4934,328
2,50,63,5575,571
3,55,63,6019,306
4,67,74,6144,207
...,...,...,...,...
1166,70,101,289377,176
1167,67,92,289396,59
1168,43,64,289398,81
1169,50,80,289400,195


In [10]:
from midi_utils import get_transformed_data

md = MidiFile('/home/ivan/Desktop/Notes/MIDI/1.mid')
data = get_transformed_data(md)
data


,note,velocity,time,dur
0,75,80,0,479
1,79,80,480,479
2,82,80,480,479
3,81,80,480,479
4,46,80,478,455
...,...,...,...,...
332,50,80,98,1821
333,58,80,0,1821
334,53,80,0,1821
335,46,80,0,1821


In [11]:
from midi_utils import load_transformed, midi_from_transformed

mid = midi_from_transformed(data)
load_transformed(data, '/home/ivan/Desktop/Notes/MIDI/at1.mid')
mid

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='PyPiano', time=0),
    MetaMessage('instrument_name', name='Steinway D Prelude', time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('set_tempo', tempo=500000, time=0),
    Message('control_change', channel=0, control=64, value=127, time=0),
    Message('note_on', channel=0, note=75, velocity=80, time=0),
    Message('note_on', channel=0, note=75, velocity=0, time=479),
    Message('note_on', channel=0, note=79, velocity=80, time=1),
    Message('note_on', channel=0, note=79, velocity=0, time=479),
    Message('note_on', channel=0, note=82, velocity=80, time=1),
    Message('note_on', channel=0, note=82, velocity=0, time=479),
    Message('note_on', channel=0, note=81, velocity=80, time=1),
    Message('note_on', channel=0, note=46, velocity=80, time=478),
    Message('note_on', channel=0, note=81

In [12]:
mid.tracks[0].__len__()

680

In [13]:
from midi_utils import form_dataset

dataset = form_dataset([
    '/home/ivan/Desktop/Notes/MIDI/1.mid',
    '/home/ivan/Desktop/Notes/MIDI/35.1.mid'
])

In [25]:
import os
midi_folder = '/home/ivan/Desktop/Notes/MIDI'
midi_files = [midi_folder+'/'+midi_file for midi_file in os.listdir(midi_folder)]
dataset = form_dataset(midi_files)

In [24]:
for file in midi_files:
    try:
        md = MidiFile(file)
    except(BaseException):
        print(file)

/home/ivan/Desktop/Notes/MIDI/Шедевр 2.mid
/home/ivan/Desktop/Notes/MIDI/FateZero_OP_-_Oath_Sign.mid


In [27]:
dataset.to_csv('all_midies.csv')

In [31]:
dataset.shape

(57717, 4)

In [29]:
load_transformed(dataset, '/home/ivan/all.mid')

# Transformer tests

In [6]:
from midi_transformer import MidiTransformer
import torch
import torch.nn as nn

from utils import batchify, device
from main import generate_square_subsequent_mask, train, generate

In [22]:
n_notes = 128
n_real_features = 2
batch_size = 1
n_epochs = 20
bptt = 10

generate_n = 50

conf = dict(
    n_notes=n_notes,
    n_real_features=n_real_features,
    d_model=10,
    nhead=2,
    d_hid=10,
    nlayers=2,
    dropout=0.0
)

model = MidiTransformer(**conf).to(device)


In [26]:
data = torch.zeros(2000, 130)
for i in range(data.shape[0]):
    data[i][i%128] = 1
data = batchify(data, batch_size)

In [27]:
for epoch in range(n_epochs):
    train(model, data, bptt, epoch)

| epoch   0 |     1/  200 batches | lr 5.00 | ms/batch 19.60 | loss  7.75 | ppl  2324.39
| epoch   0 |     2/  200 batches | lr 5.00 | ms/batch  9.99 | loss  2.42 | ppl    11.26
| epoch   0 |     3/  200 batches | lr 5.00 | ms/batch 11.69 | loss  3.12 | ppl    22.55
| epoch   0 |     4/  200 batches | lr 5.00 | ms/batch  8.89 | loss  1.55 | ppl     4.71
| epoch   0 |     5/  200 batches | lr 5.00 | ms/batch 10.93 | loss  1.91 | ppl     6.76
| epoch   0 |     6/  200 batches | lr 5.00 | ms/batch  6.78 | loss  0.81 | ppl     2.26
| epoch   0 |     7/  200 batches | lr 5.00 | ms/batch  9.42 | loss  0.94 | ppl     2.56
| epoch   0 |     8/  200 batches | lr 5.00 | ms/batch  6.23 | loss  0.32 | ppl     1.37
| epoch   0 |     9/  200 batches | lr 5.00 | ms/batch  8.42 | loss  0.38 | ppl     1.47
| epoch   0 |    10/  200 batches | lr 5.00 | ms/batch  6.54 | loss  0.12 | ppl     1.13
| epoch   0 |    11/  200 batches | lr 5.00 | ms/batch  6.83 | loss  0.22 | ppl     1.25
| epoch   0 |    12/ 

KeyboardInterrupt: 

In [25]:
generate(model, data[1:2, 0], 20)

array([[ 1.,  0.,  0.],
       [ 2.,  0.,  0.],
       [ 3.,  0.,  0.],
       [ 4.,  0.,  0.],
       [ 5.,  0.,  0.],
       [ 6.,  0.,  0.],
       [ 7.,  0.,  0.],
       [ 8.,  0.,  0.],
       [ 9.,  0.,  0.],
       [10.,  0.,  0.],
       [11.,  0.,  0.],
       [12.,  0.,  0.],
       [13.,  0.,  0.],
       [14.,  0.,  0.],
       [15.,  0.,  0.],
       [16.,  0.,  0.],
       [64.,  0.,  0.],
       [39.,  0.,  0.],
       [40.,  0.,  0.],
       [41.,  0.,  0.]], dtype=float32)